# HTML table to Pandas Data Frame to Portal Item

This notebooks shows how Pandas can be used to extract data from a table within a web page (in this case, a Wikipedia article) and how it can be then brought into the GIS for further analysis and visualization.

In [1]:
import pandas as pd

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/Number_of_guns_per_capita_by_country")[2]

In [3]:
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))

In [4]:
df.head()

,Country,Guns per 100 Residents,Rank,Notes
1,United States,112.6[6],1,According to the Congressional Research Servic...
2,Serbia,75.6,2,NaN
3,Yemen,54.8,3,NaN
4,Switzerland,45.7[note 1],4,Including the personal weapons of the militia....
5,Cyprus,36.4[9],5,NaN


In [5]:
df.iloc[0,1] = 112.6

In [6]:
import matplotlib.pyplot as plt

In [7]:
df.dtypes

0
Country                   object
Guns per 100 Residents    object
Rank                      object
Notes                     object
dtype: object

In [8]:
df = df.convert_objects(convert_numeric=True)

In [9]:
df.head(15)

,Country,Guns per 100 Residents,Rank,Notes
1,United States,112.6,1,According to the Congressional Research Servic...
2,Serbia,75.6,2,NaN
3,Yemen,54.8,3,NaN
4,Switzerland,NaN,4,Including the personal weapons of the militia....
5,Cyprus,NaN,5,NaN
6,Saudi Arabia,35.0,6,NaN
7,Iraq,34.2,7,NaN
8,Uruguay,31.8,8,NaN
9,Sweden,31.6,9,According to the Swedish National Police Agenc...
10,Norway,31.3,10,NaN


In [10]:
from arcgis.gis import GIS
from arcgis.lyr import FeatureCollection
from arcgis.viz import MapView
import json

gis = GIS("https://deldev.maps.arcgis.com", "demo_deldev", "P@ssw0rd")

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [11]:
path = "content/features/analyze"
postdata = {
    "f": "pjson",
    "text" : df.to_csv(),
    "filetype" : "csv",
    "analyzeParameters" : {
        "enableGlobalGeocoding": "true",
        "sourceLocale":"en-us",
        "locationType":"address",
        "sourceCountry":"",
        "sourceCountryHint":""
    }
}


res = gis._portal.con.post(path, postdata)
#import json
#json.dumps(res)
res['publishParameters'].update({"addressFields":{"CountryCode":"Country"}})

path = "content/features/generate"
postdata = {
    "f": "pjson",
    "text" : df.to_csv(),
    "filetype" : "csv",
    "publishParameters" : json.dumps(res['publishParameters'])
}

res2014 = gis._portal.con.post(path, postdata,  use_ordered_dict=True)
#print(json.dumps(res2014))
fc2014 = FeatureCollection(res2014['featureCollection']['layers'][0])

In [12]:
map = gis.map('USA', 2)

In [13]:
map

<img src="http://esri.github.io/arcgis-python-api/notebooks/nbimages/wikidata.png"/>

In [14]:
map.add_layer(fc2014)

In [15]:
item_properties = {
    "title": "Worldwide gun ownership",
    "tags" : "guns,violence",
    "snippet": " GSR Worldwide gun ownership",
    "description": "test description",
    "text": json.dumps({"featureCollection": {"layers": [ fc2014 ]}}),
    "type": "Feature Collection",
    "typeKeywords": "Data, Feature Collection, Singlelayer",
    "extent" : "-102.5272,-41.7886,172.5967,64.984"
}
item = gis.content.add(item_properties)


In [16]:
item

<Item title:"Worldwide gun ownership" type:Feature Collection owner:demo_deldev>